## Setting up the development environment

In [1]:
!pip install cohere tiktoken openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00


## Loading data

In [2]:
!pip install python-dotenv

In [3]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_key  = os.environ["OPENAI_API_KEY"]

In [4]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

file_name = "fictitious_movie_data.csv"

df = pd.read_csv(file_name)
df.shape

(120, 8)

In [5]:
df.sample(3)

,Movie Title,Description,Director,Star Rating,Critic Review 1,Critic Review 2,Critic Review 3,Synopsis
30,Echoes in the Glass,A bartender experiences the same day over and over until he realizes the key to break the cycle.,Carl Johnson,4.1,"A clever take on the time-loop genre, Carl Johnson keeps the narrative fresh and engaging throughout.",Echoes in the Glass could have easily fallen into repetition but manages to keep the viewer hooked.,"Johnson deftly mixes comedy, drama, and a dash of existential crisis in a compelling cocktail of a film.","Eddie is the head bartender at ""The Timepiece,"" a cozy but always bustling pub in the heart of a vibrant city. Known for his impeccable skill in crafting cocktails and his cheery disposition, he has a regular clientele who adore him. The film begins on what seems to be an ordinary day, with Eddie heading to work and engaging in familiar banter with patrons.\n\nBut as the day progresses, peculiar incidents arise. A couple argues about a misplaced engagement ring, a businessman forgets his briefcase, and a street musician plays an oddly haunting tune outside. By the end of the night, a mysterious woman in a crimson dress, whom Eddie doesn't recognize, leaves a note with the words, ""The key is within you.""\n\nThe following day, Eddie wakes up to an unsettling déjà vu. The same couple argues, the businessman leaves his briefcase again, and the street musician plays that haunting tune. It doesn't take long for Eddie to realize he's reliving the exact same day.\n\nPanic sets in during the subsequent repetitions. Eddie tries varying things, from confronting the arguing couple to safeguarding the businessman's briefcase, but each attempt only leads to the same conclusion: the note from the woman in the crimson dress.\n\nEddie becomes obsessed with deciphering the message, believing it to be the key to escaping the loop. He confronts the woman, but she remains enigmatic, offering only riddles as clues. Days turn into weeks, weeks into months, all the same, and Eddie's once cheery demeanor fades into desolation.\n\nOne iteration, in a moment of clarity, Eddie begins to focus on his own actions. Instead of being reactive, he starts being proactive. He sits down with the arguing couple, not to discuss the ring, but to delve deeper. He learns of their insecurities and fears, offering counsel that helps them rediscover their love. The businessman, it turns out, is on the brink of a breakdown from work stress. Eddie convinces him to take a day off and enjoy life, changing the man's perspective. The street musician, struggling with a lost love, finds solace in Eddie's kind words and plays a new, hopeful tune.\n\nYet, the loop persists.\n\nIn a profound moment of introspection, Eddie recalls a tragic incident from his past. A car accident years prior had claimed the life of his younger sister, Lily. Consumed by guilt (he was driving), Eddie had buried the memory deep, using his work as a distraction. The Timepiece, with its endless array of customers and their stories, became his escape.\n\nThe next day, or rather, the same day once more, Eddie visits Lily's grave, a place he had avoided for years. He pours out his heart, finally confronting his guilt and grief. As the sun sets, the woman in the crimson dress appears, revealing herself to be a manifestation of Eddie's subconscious. She congratulates him on finding the key—the acceptance of his past and the release of his guilt.\n\nEddie wakes up the next morning to the sound of rain—a deviation from the sunny days he had been experiencing. He realizes the cycle has broken. The Timepiece is different too, quieter, giving Eddie a chance to reflect and mourn properly.\n\nAs the film concludes, Eddie, while still a bartender, becomes a pillar of support for many. But now, it’s not to distract himself from his pain, but to genuinely connect and help heal others, just as he has healed.\n\n""Echoes in the Glass"" is a poignant exploration of grief, guilt,

In [6]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

# Define prompt
prompt_template = """
Your task is to generate a brief summary of movie synopsis.
The Synopsis is delimited by triple backticks.

Summarize with a focus on people that mentioned in the content.
Start your response with the title of the movie.
Be concise.
The output text should be very short using no more than 30 words.

Your audiences are 9-year-olds.

Synopsis: ```{synopsis}```
"""

In [7]:
llm_name = "gpt-3.5-turbo"

llm = ChatOpenAI(model_name=llm_name, temperature=0)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

In [8]:
synopsis = df.iloc[1]['Movie Title']+'. '+df.iloc[1]['Synopsis']
synopsis[:100]

'Love in Venice. Set amidst the enchanting canals and historic palazzos of Venice, Love in Venice is '

In [9]:
result = llm_chain.predict(synopsis=synopsis)
result

'Love in Venice: Isabella, a violinist, and Alexei, an architect, discover a hidden love story while trying to save a concert hall. They unite to preserve the hall and find love themselves.'

## Running over a dataframe

In [10]:
df_sub = df.head(30)
results = []

import time
start = time.perf_counter()

for index, row in df_sub.iterrows():
    result = llm_chain.predict(synopsis=row['Movie Title']+'. '+row['Synopsis'])
    results.append(result)

elapsed = time.perf_counter() - start

print(f"Serial executed in {elapsed:0.2f} seconds.")

Serial executed in 121.40 seconds.


In [11]:
results

['Galactic Odyssey: Captain Lana Veridi and her crew embark on a mission to save Earth by finding the Celestial Compass, facing alternate realities and sacrificing memories along the way.',
 'Love in Venice: Isabella, a violinist, and Alexei, an architect, discover a hidden love story while trying to save a concert hall. They unite to preserve the hall and find love themselves.',
 'Title: Quantum Nexus\n\nDr. Elara Bishop discovers the Quantum Nexus, a device that allows travel between parallel universes. She teams up with Max and Lena to save an alternate Earth from destruction, facing adversaries and ethical dilemmas along the way. Elara must make a heart-wrenching decision to ensure the safety of her reality.',
 'Title: Midnight Mirage\n\nSynopsis: Kara, a journalist, encounters the Midnight Mirage, a trapped soul who grants glimpses of alternate lives. Determined to free him, she embarks on a perilous journey with her old flame, Alex. In a heart-rending sacrifice, they break the cu

In [12]:
df_sub['Summary'] = results
df_sub[['Movie Title','Director','Synopsis','Summary']]

<ipython-input-12-901e9fda22cf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['Summary'] = results


,Movie Title,Director,Synopsis,Summary
0,Galactic Odyssey,Alex Mercer,"In the year 3078, amidst the backdrop of a sprawling intergalactic civilization, Captain Lana Veridi and her diverse crew aboard the starship Odyssey are renowned for exploring uncharted territories of space. However, their latest mission isn’t one of discovery, but of desperate necessity. A mysterious cosmic event known as The Pulse has begun to distort time and reality on Earth, causing it to merge and split with alternate realities.\n\nThe Odyssey is tasked with traveling to the edge of the known universe to find the Celestial Compass, a legendary artifact said to possess the power to realign time and dimensions. Skeptical but undeterred, Captain Veridi embarks on the mission, aware of the rising stakes. As Earth's very existence begins to crumble, visions of alternate Earths become more frequent, giving glimpses of both wonders and horrors.\n\nAs the crew journey deeper into the unknown, they encounter remnants of ancient civilizations, cosmic entities, and clues pointing to a long-forgotten empire that once attempted to harness the power of the Celestial Compass, only to vanish without a trace. Along the way, the crew must confront their own alternate selves, facing choices and paths they once took or might have taken, challenging their beliefs and convictions.\n\nRomance blossoms between Lana and the ship’s historian, Dr. Elian Korr, who harbors secrets of his own. A former member of a sect devoted to protecting the secrets of the Celestial Compass, Elian struggles with his loyalties, eventually revealing that the Compass isn’t just a tool, but a sentient being that chooses its wielder.\n\nWhen they finally locate the Compass at the nexus of a cosmic storm, it communicates telepathically with Lana, revealing that The Pulse was a call for help, a beacon sent out to find someone worthy of wielding its power. But the choice comes with a price: to save her Earth, Lana must let go of the personal memories that anchor her to it.\n\nAs the climax unfolds, Lana, with the support of Elian and her crew, decides to merge her consciousness with the Compass. She becomes the beacon that realigns Earth's reality, sacrificing her memories to save humanity.\n\nThe film closes with a bittersweet victory. Earth is saved, with its inhabitants none the wiser of the crisis they averted. Aboard the Odyssey, the crew mourns the loss of the Lana they knew, even as they welcome her back. The Captain has no recollection of her past but is surrounded by a crew that promises to help her build new memories.\n\nGalactic Odyssey is a cosmic tale of sacrifice, love, and the intricate dance of destiny, set against the vast, awe-inspiring backdrop of the universe.","Galactic Odyssey: Captain Lana Veridi and her crew embark on a mission to save Earth by finding the Celestial Compass, facing alternate realities and sacrificing memories along the way."
1,Love in Venice,Sophia Anderson,"Set amidst the enchanting canals and historic palazzos of Venice, Love in Venice is a contemporary romantic drama that weaves together the lives of two strangers bound by fate and a shared past.\n\nIsabella Martelli, a talented violinist, returns to her hometown of Venice after a decade of performing around the world. The weight of her family's legacy, the renowned Martelli Orchestra, rests on her shoulders as she prepares for a grand concert meant to revive the slowly fading orchestra and save their historic concert hall from being sold to a developer.\n\nEnter Alexei Ivanov, a world-famous architect known for his modernist creations, but with a secret passion for historical restorations. He's in Venice on a lucrative assignment to redesign the Venetian waterfront, which includes the Martelli's ancestral concert hall. Despite the seemingly opposing goals, there's an undeniable chemistry between Isabella and Alexei from their first accidental meeting at Piazza San Marco.\n\nAs they spend more time together,

## Asynchronous calls to the API
So instead of waiting Idly for the response, what if we sent all the calls to the API at the same time? This way we would only have to wait for a single response and then process them. This is called Asynchronous calls to the API.



In [13]:
async def async_generate(chain, inputs, unique_id):
    """
    Asynchronous task to extract summary
    """
    response = await chain.arun(inputs)
    return response

In [14]:
import asyncio

async def generate_concurrently(df, llm_chain):
    for attempt in range(3):
        try:
            tasks = []
            for _, row in df.iterrows():
                synopsis = row['Movie Title']+'. '+row['Synopsis']
                unique_id = row['Movie Title']

                tasks.append(async_generate(llm_chain, synopsis, unique_id))

            results = await asyncio.gather(*tasks)
            return results
        except openai.error.RateLimitError as e:
            print("rate limit error. retrying ...")
            await asyncio.sleep(60)

    print("Maximum retries reached for rate limit error.")

In [15]:
start = time.perf_counter()

ans = await generate_concurrently(df_sub, llm_chain)

elapsed = time.perf_counter() - start

print(f"Concurrent executed in {elapsed:0.2f} seconds.")

Concurrent executed in 5.81 seconds.


In [16]:
results

['Galactic Odyssey: Captain Lana Veridi and her crew embark on a mission to save Earth by finding the Celestial Compass, facing alternate realities and sacrificing memories along the way.',
 'Love in Venice: Isabella, a violinist, and Alexei, an architect, discover a hidden love story while trying to save a concert hall. They unite to preserve the hall and find love themselves.',
 'Title: Quantum Nexus\n\nDr. Elara Bishop discovers the Quantum Nexus, a device that allows travel between parallel universes. She teams up with Max and Lena to save an alternate Earth from destruction, facing adversaries and ethical dilemmas along the way. Elara must make a heart-wrenching decision to ensure the safety of her reality.',
 'Title: Midnight Mirage\n\nSynopsis: Kara, a journalist, encounters the Midnight Mirage, a trapped soul who grants glimpses of alternate lives. Determined to free him, she embarks on a perilous journey with her old flame, Alex. In a heart-rending sacrifice, they break the cu

In [17]:
df_sub['Async_Summary'] = results
df_sub[['Movie Title','Director','Synopsis','Summary','Async_Summary']]

<ipython-input-17-3205ae1392b9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['Async_Summary'] = results


,Movie Title,Director,Synopsis,Summary,Async_Summary
0,Galactic Odyssey,Alex Mercer,"In the year 3078, amidst the backdrop of a sprawling intergalactic civilization, Captain Lana Veridi and her diverse crew aboard the starship Odyssey are renowned for exploring uncharted territories of space. However, their latest mission isn’t one of discovery, but of desperate necessity. A mysterious cosmic event known as The Pulse has begun to distort time and reality on Earth, causing it to merge and split with alternate realities.\n\nThe Odyssey is tasked with traveling to the edge of the known universe to find the Celestial Compass, a legendary artifact said to possess the power to realign time and dimensions. Skeptical but undeterred, Captain Veridi embarks on the mission, aware of the rising stakes. As Earth's very existence begins to crumble, visions of alternate Earths become more frequent, giving glimpses of both wonders and horrors.\n\nAs the crew journey deeper into the unknown, they encounter remnants of ancient civilizations, cosmic entities, and clues pointing to a long-forgotten empire that once attempted to harness the power of the Celestial Compass, only to vanish without a trace. Along the way, the crew must confront their own alternate selves, facing choices and paths they once took or might have taken, challenging their beliefs and convictions.\n\nRomance blossoms between Lana and the ship’s historian, Dr. Elian Korr, who harbors secrets of his own. A former member of a sect devoted to protecting the secrets of the Celestial Compass, Elian struggles with his loyalties, eventually revealing that the Compass isn’t just a tool, but a sentient being that chooses its wielder.\n\nWhen they finally locate the Compass at the nexus of a cosmic storm, it communicates telepathically with Lana, revealing that The Pulse was a call for help, a beacon sent out to find someone worthy of wielding its power. But the choice comes with a price: to save her Earth, Lana must let go of the personal memories that anchor her to it.\n\nAs the climax unfolds, Lana, with the support of Elian and her crew, decides to merge her consciousness with the Compass. She becomes the beacon that realigns Earth's reality, sacrificing her memories to save humanity.\n\nThe film closes with a bittersweet victory. Earth is saved, with its inhabitants none the wiser of the crisis they averted. Aboard the Odyssey, the crew mourns the loss of the Lana they knew, even as they welcome her back. The Captain has no recollection of her past but is surrounded by a crew that promises to help her build new memories.\n\nGalactic Odyssey is a cosmic tale of sacrifice, love, and the intricate dance of destiny, set against the vast, awe-inspiring backdrop of the universe.","Galactic Odyssey: Captain Lana Veridi and her crew embark on a mission to save Earth by finding the Celestial Compass, facing alternate realities and sacrificing memories along the way.","Galactic Odyssey: Captain Lana Veridi and her crew embark on a mission to save Earth by finding the Celestial Compass, facing alternate realities and sacrificing memories along the way."
1,Love in Venice,Sophia Anderson,"Set amidst the enchanting canals and historic palazzos of Venice, Love in Venice is a contemporary romantic drama that weaves together the lives of two strangers bound by fate and a shared past.\n\nIsabella Martelli, a talented violinist, returns to her hometown of Venice after a decade of performing around the world. The weight of her family's legacy, the renowned Martelli Orchestra, rests on her shoulders as she prepares for a grand concert meant to revive the slowly fading orchestra and save their historic concert hall from being sold to a developer.\n\nEnter Alexei Ivanov, a world-famous architect known for his modernist creations, but with a secret passion for historical restorations. He's in Venice on a lucrative assignment to redesign the Venetian waterfront, which includes the Martelli's ancestral con